In [1]:
import os
import copy

import torch
import torch.optim as optim
import tensorly as tl

from tddl.models.resnet import PA_ResNet18
from tddl.models.resnet_lr import low_rank_resnet18
from tddl.models.utils import count_parameters
from tddl.factorizations import factorize_network, number_layers

/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.6/site-packages/tensorly/backend/pytorch_backend.py:200: UserWarning: You are using an old version of PyTorch (1.7.1+cu101). We recommend upgrading to a newest one, e.g. >1.8.0.
  warnings.warn(f'You are using an old version of PyTorch ({torch.__version__}). '


In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
tl.set_backend('pytorch')

cuda = "0"
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = cuda

cpu = "2"
os.environ["MKL_NUM_THREADS"] = cpu
os.environ["NUMEXPR_NUM_THREADS"] = cpu
os.environ["OMP_NUM_THREADS"] = cpu

In [4]:
!pwd

/home/jetzeschuurman/gitProjects/phd/tddl/notebooks


In [5]:
save_path = "/home/jetzeschuurman/gitProjects/phd/tddl/notebooks/tmp"

In [3]:
pretrained = "/local/jetzeschuurman/f_mnist/logs/parn_18_d0.5_256_sgd_l0.1_g0.1_sTrue/1633280228/cnn_best"

# load pretrained model
pretrained_model = torch.load(pretrained)

In [7]:
pre_param = count_parameters(pretrained_model)
pre_param

11170122

In [8]:
number_layers(pretrained_model)

{'conv1': (0,
  Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
 'bn1': (1,
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
 'layer1': (2,
  {'0': (3,
    {'bn1': (4,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv1': (5,
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'bn2': (6,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv2': (7,
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'shortcut': (8, Sequential())}),
   '1': (9,
    {'bn1': (10,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv1': (11,
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'bn2': (12,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
 

In [9]:
fact_model = copy.deepcopy(pretrained_model)

# TODO: do I also consider the skip conneciton layers?
# For now not
layers = [5, 7, 11, 13, 18, 20, 25, 27, 32, 34, 39, 41, 46, 48, 53, 55]
factorization='tucker'
rank=0.5
decompose_weights=True

decomposition_kwargs = {'init': 'random'} if factorization == 'cp' else {}
fixed_rank_modes = 'spatial' if factorization == 'tucker' else None

output = factorize_network(
    fact_model,
    layers=layers,
    factorization=factorization,
    rank=rank,
    decompose_weights=decompose_weights,
    return_error=True,
    verbose=True,
)

0 conv1 <class 'torch.nn.modules.conv.Conv2d'>
1 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
2 layer1 <class 'torch.nn.modules.container.Sequential'>
3 0 <class 'tddl.models.resnet.PreActBlock'>
4 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
5 conv1 <class 'torch.nn.modules.conv.Conv2d'>
6 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
7 conv2 <class 'torch.nn.modules.conv.Conv2d'>


/home/jetzeschuurman/gitProjects/phd/tddl/venv/lib/python3.6/site-packages/tensorly/backend/core.py:885: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


8 shortcut <class 'torch.nn.modules.container.Sequential'>
9 1 <class 'tddl.models.resnet.PreActBlock'>
10 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
11 conv1 <class 'torch.nn.modules.conv.Conv2d'>
12 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
13 conv2 <class 'torch.nn.modules.conv.Conv2d'>
14 shortcut <class 'torch.nn.modules.container.Sequential'>
15 layer2 <class 'torch.nn.modules.container.Sequential'>
16 0 <class 'tddl.models.resnet.PreActBlock'>
17 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
18 conv1 <class 'torch.nn.modules.conv.Conv2d'>
19 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
20 conv2 <class 'torch.nn.modules.conv.Conv2d'>
21 shortcut <class 'torch.nn.modules.container.Sequential'>
22 0 <class 'torch.nn.modules.conv.Conv2d'>
23 1 <class 'tddl.models.resnet.PreActBlock'>
24 bn1 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
25 conv1 <class 'torch.nn.modules.conv.Conv2d'>
26 bn2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
27 con

In [35]:
output

{'conv1': (0,
  None,
  Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
 'bn1': (1,
  None,
  BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
 'layer1': (2,
  None,
  {'0': (3,
    None,
    {'bn1': (4,
      None,
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv1': (5,
      tensor(1.9908, device='cuda:0', grad_fn=<CopyBackwards>),
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'bn2': (6,
      tensor(1.9908, device='cuda:0', grad_fn=<CopyBackwards>),
      BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
     'conv2': (7,
      tensor(3.0027, device='cuda:0', grad_fn=<CopyBackwards>),
      Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)),
     'shortcut': (8,
      tensor(3.0027, device='cuda:0', grad_fn=<CopyBackwards>),
      Sequential())}),
   '1': (9,
    None

In [58]:
def outer(output, layers):
    list_errors = []
    
    def parse_errors(d, layers, 
        # list_errors=None,
    ):
        nonlocal list_errors
        # if list_errors is None:
        #     list_errors = []

        for k, v in d.items():
            # print(v[0])
            if isinstance(v[2], dict):
                parse_errors(v[2], layers)
            elif v[0] in layers:
                print(k,v)
                list_errors.append(v)

    parse_errors(output, layers)
    return list_errors

list_errors = outer(output, layers)

conv1 (5, tensor(1.9908, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
conv2 (7, tensor(3.0027, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
conv1 (11, tensor(3.4912, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
conv2 (13, tensor(3.5323, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
conv1 (18, tensor(3.6274, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False))
conv2 (20, tensor(4.5748, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))
conv1 (25, tensor(5.0622, device='cuda:0', grad_fn=<CopyBackwards>), Conv2d(128, 128, kernel_size=(3, 3), strid

In [60]:
list_errors[0]

(5,
 tensor(1.9908, device='cuda:0', grad_fn=<CopyBackwards>),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False))

In [17]:
tuckertensor = fact_model.layer1[0].conv1.weight
tuckertensor

TuckerTensor(shape=(64, 64, 3, 3), rank=(39, 39, 3, 3))

In [21]:
approx = tuckertensor.to_tensor()
approx.shape

torch.Size([64, 64, 3, 3])

In [22]:
tensor = pretrained_model.layer1[0].conv1.weight
tensor.shape

torch.Size([64, 64, 3, 3])

In [23]:
torch.norm(approx-tensor)

tensor(1.9908, device='cuda:0', grad_fn=<CopyBackwards>)

In [ ]:
import torch

def calculate_error(
    original, 
    approximation,
    **kwargs,
):
    return torch.norm(original-approximation, **kwargs)

In [2]:
import json

errors_path = "/local/jetzeschuurman/f_mnist/logs/erros.json"

with open(errors_path) as f:
    errors = json.load(f)

In [3]:
errors

[[5,
  0.16598109900951385,
  'Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [7,
  0.25764283537864685,
  'Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [11,
  0.2802920341491699,
  'Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [13,
  0.2963773012161255,
  'Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [18,
  0.26150768995285034,
  'Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)'],
 [20,
  0.24951869249343872,
  'Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [25,
  0.3009827733039856,
  'Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [27,
  0.3131818175315857,
  'Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)'],
 [32,
  0.31080150604248047,
  'Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1,

In [6]:
import yaml
from pathlib import Path

yaml_path = Path("/home/jetzeschuurman/gitProjects/phd/tddl/configs/factorize.yml")
config_data = yaml.load(yaml_path.read_text(), Loader=yaml.Loader)

In [10]:
if config_data['decompose_weights']:
    print('y')
else:
    print('else')

else


pretrained_model

In [22]:

mpath = [2, 0, 2]

def get_module(model, mpath):
    cmod = model
    for p in mpath:
        cs = list(cmod.named_children())
        cmod = cs[p][1]
    return cmod

cmod = get_module(pretrained_model, mpath[:-1])
print(cmod)






PreActBlock(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (shortcut): Sequential()
)


In [ ]:
m